# TODO

## finalize transmission module

In [1]:
import os
import pkg_resources

import leafmap
import whitebox
import rasterio
import yaml

import numpy as np
import geopandas as gpd

from rasterio import features

import cerf

wbt = whitebox.WhiteboxTools()


In [5]:
data_dir = '/Users/d3y010/projects/cerf/data'

states_file = pkg_resources.resource_filename('cerf', 'data/state-abbrev_to_state-name.yml')

original_file = os.path.join(data_dir, 'hifld', 'Electric_Substations-shp', 'Substations.shp')

template_raster = pkg_resources.resource_filename('cerf', 'data/cerf_conus_states_albers_1km.tif')

substation_raster = '/Users/d3y010/Desktop/subs_distance.tif'

natgas_file = os.path.join(data_dir, 'eia', 'NaturalGas_InterIntrastate_Pipelines_US_EIA', 'NaturalGas_Pipelines_US_202001.shp')


In [3]:
gdf = cerf.process_hifld_substations()


In [4]:
cerf.transmission_to_distance_raster(gdf, substation_raster)


In [6]:
xdf = gpd.read_file(natgas_file)

xdf


In [9]:
xdf = g

Index(['TYPEPIPE', 'Operator', 'Status', 'Shape_Leng', 'geometry'], dtype='object')

In [8]:

xdf.TYPEPIPE.unique()


array(['Intrastate', 'Interstate', 'Gathering'], dtype=object)

In [10]:
xdf.Status.unique()


array(['Operating'], dtype=object)

In [ ]:
cerf.transmission_to_distance_raster(xdf, '/Users/d3y010/Desktop/projects')


In [11]:
with rasterio.open(template_raster) as src:
    target_crs = src.crs


In [13]:
with open(states_file, 'r') as yml:
    states = yaml.load(yml, Loader=yaml.FullLoader)


In [14]:
gdf = gpd.read_file(original_file)


In [15]:
# keep only substations in the CONUS that are either in service or under construction
gdf = gdf.loc[(gdf['TYPE'] == 'SUBSTATION') & 
              (gdf['STATE'].isin(states.keys())) &
              (gdf['STATUS'].isin(('IN SERVICE', 'UNDER CONST')))].copy()

# set field to be used as raster value
gdf['value'] = 1


In [12]:
xdf = gdf[['geometry']].copy()
xdf = xdf.to_crs(target_crs)


xdf.to_file('/Users/d3y010/repos/github/cerf/cerf/data/hifld_substations_conus_albers.shp')


In [16]:
with rasterio.open(template_raster) as src:
    
    # create 0 where land array
    arr = src.read(1) * 0
    
    metadata = src.meta.copy()
    target_crs = src.crs
    
    with rasterio.open(substation_raster, 'w', **metadata) as out:

        # reproject transmission data
        gdf = gdf.to_crs(target_crs)

        # get shapes
        shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf['value']))

        # burn shapes
        burned = features.rasterize(shapes=shapes, fill=0, out=arr, transform=out.transform)

        out.write_band(1, burned)


In [46]:
from rasterio.io import MemoryFile
from rasterio.plot import show
import tempfile


def suppress_callback(value):
    """Do not log callback output for whitebox functions."""
    
    pass



with tempfile.NamedTemporaryFile(suffix='.tif') as temp:

    with rasterio.open(template_raster) as src:

        # create 0 where land array
        arr = src.read(1) * 0

        metadata = src.meta.copy()
        
        # reproject transmission data
        gdf = gdf.to_crs(src.crs)

        # get shapes
        shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf['_rval_']))

    # rasterize transmission vector data and save to memory
    with rasterio.open(temp.name, 'w', **metadata) as dataset:

        # burn features into raster
        burned = features.rasterize(shapes=shapes, fill=0, out=arr, transform=dataset.transform)

        # write the outputs to file
        dataset.write_band(1, burned)

    # calculate Euclidean distance file and write raster; result just stores the return value 0
    result = wbt.euclidean_distance(temp.name, substation_raster, callback=suppress_callback)



In [30]:
dataset.

['/vsimem/b8cb4085-dcb9-4f53-bbd4-2171a4a98df5/b8cb4085-dcb9-4f53-bbd4-2171a4a98df5.tif']

In [22]:
t = tempfile.NamedTemporaryFile()


In [23]:
print(t.name)
t.close()

/var/folders/1q/kbqf0s_90m907n320mhb_gq46y9fxr/T/tmpnv4hwuep


In [25]:

def suppress_callback(value):
    """Do not log callback output for whitebox functions."""
    
    pass

# value of v will be 0 if succ
v = wbt.euclidean_distance(
    substation_raster, 
    '/Users/d3y010/Desktop/euclidean_distance.tif',
    callback=suppress_callback
)


In [ ]:
output_units='km'

    :param output_units:                        Units of distance. Default is kilometers (km).  Alternate options
                                                are miles (mile) and meters (m).
    :type output_units:                         str
            
            
# get conversion factor for native units of meters
units_lowercase = output_units.lower()
if units_lowercase == 'km':
    conversion_factor = 0.001
elif units_lowercase == 'mile':
    conversion_factor = 0.000621371
elif units_lowercase == 'm':
    conversion_factor = 1
else:
    raise KeyError(f"'output_units' must be in 'km' or 'mile'. Option '{output_units}' not supported.")


## finalize lmp module

## finalize read the docs

## regenerate figure for paper

## metarepo

## finalize tutorial

## cerf on pypy

In [1]:
import os

import rasterio


In [3]:
sdf = gpd.read_file('/Users/d3y010/projects/cerf/data/eia/shp/cerf_substations_clean.shp')

In [4]:
sdf.head()

,NAME,TYPE,STATUS,MAX_VOLT,MIN_VOLT,geometry
0,UNKNOWN107655,SUBSTATION,IN SERVICE,161.0,69.0,POINT (-10226336.565 5743316.246)
1,UNKNOWN107656,SUBSTATION,IN SERVICE,115.0,69.0,POINT (-10053464.958 5706699.742)
2,UNKNOWN107657,SUBSTATION,IN SERVICE,115.0,-999999.0,POINT (-10144276.066 5656784.281)
3,SOUTH SHEBOYGAN FALLS,SUBSTATION,IN SERVICE,138.0,69.0,POINT (-9778125.514 5420826.129)
4,CHALK HILLS HY,SUBSTATION,IN SERVICE,138.0,-999999.0,POINT (-9773953.789 5702771.663)
